# Neural Network Classifier 

We are going to build a PyTorch neural network classifier using scRNAseq data. 

Helpful link:
Interfacing pytorch models with anndata: https://anndata.readthedocs.io/en/latest/tutorials/notebooks/annloader.html

Possibly helpful about speeding up pytorch modeling: https://sebastianraschka.com/blog/2023/pytorch-faster.html

This program will:

- pull in scRNAseq data from cell_census
- wrangle the data for modelling
- appply the NN

At first, we're just going to pick some data from cell_census. Once we get the basics we will modify to select the exact data we want.


In [11]:
import cell_census
import anndata as ad

import torch
import torch.nn

from sklearn.preprocessing import LabelEncoder

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Read in the data

We are somewhat randomly selecting the data now, but will come back and update this later. Let's just get something working.

In [3]:
census = cell_census.open_soma(census_version="latest")


In [4]:
adata = cell_census.get_anndata(
        census=census,
        organism = "Homo sapiens",
        obs_value_filter = 'tissue_ontology_term_id == "UBERON:0002299" and assay == "10x 3\' v3"',
        column_names={"obs": ["sex"]},
        )

display(adata)

AnnData object with n_obs × n_vars = 57747 × 60664
    obs: 'sex', 'tissue_ontology_term_id', 'assay'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

In [5]:
# the data is stored in adata.X
adata.X

<57747x60664 sparse matrix of type '<class 'numpy.float32'>'
	with 136730898 stored elements in Compressed Sparse Row format>

In [15]:
# select the labels. 
labels = adata.obs

In [16]:
labels.head()

,sex,tissue_ontology_term_id,assay
0,male,UBERON:0002299,10x 3' v3
1,male,UBERON:0002299,10x 3' v3
2,male,UBERON:0002299,10x 3' v3
3,male,UBERON:0002299,10x 3' v3
4,male,UBERON:0002299,10x 3' v3


In [19]:
lb = LabelEncoder()
labels['encoded_labels'] = lb.fit_transform(labels['sex'])
labels.head(5)

,sex,tissue_ontology_term_id,assay,encoded_labels
0,male,UBERON:0002299,10x 3' v3,1
1,male,UBERON:0002299,10x 3' v3,1
2,male,UBERON:0002299,10x 3' v3,1
3,male,UBERON:0002299,10x 3' v3,1
4,male,UBERON:0002299,10x 3' v3,1
